# **MACHINE LEARNING**

##Tugas Praktikum Modul 06 - Ensemble Learning


---


##Masfudin Dwi Cahyo
##2141720010
##TI-3D


---



# **Tugas 2**

Terdapat dataset mushroom. Berdasarkan dataset tersebut, bandingkan peforma antara algoritma Decision Tree dan AdaBoost. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.



---



**Import Library**

In [68]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import AdaBoostClassifier # import AdaBoost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder # import LabelEncoder

**Load Data**

In [69]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning/Week 06/Tugas Praktikum/mushrooms.csv')

df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [71]:
# Cek kolom null
df.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

**Seleksi Fitur**

In [72]:
# Seleksi fitur
X = df.drop('class', axis=1)  # fitur class di-drop karena hanya mendefinisikan jamur dapat dimakan atau tidak, axis=1 menandakan arah horizontal
y = df['class']  # fitur class sebagai target, karena fitur class di-drop, data 'p' dan 'e' di dalamnya tidak perlu di encode

# membuat model Label Encoder
le = LabelEncoder()

# encode fungsi/kolom
for col in X.columns:
  if X[col].dtype == 'object':
    X[col] = le.fit_transform(X[col])

# Cek jumlah fitur dan instance
print(X.shape)
X.info()
y.info()

(8124, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   cap-shape                 8124 non-null   int64
 1   cap-surface               8124 non-null   int64
 2   cap-color                 8124 non-null   int64
 3   bruises                   8124 non-null   int64
 4   odor                      8124 non-null   int64
 5   gill-attachment           8124 non-null   int64
 6   gill-spacing              8124 non-null   int64
 7   gill-size                 8124 non-null   int64
 8   gill-color                8124 non-null   int64
 9   stalk-shape               8124 non-null   int64
 10  stalk-root                8124 non-null   int64
 11  stalk-surface-above-ring  8124 non-null   int64
 12  stalk-surface-below-ring  8124 non-null   int64
 13  stalk-color-above-ring    8124 non-null   int64
 14  stalk-color-below-ring    812

**Split Data Training dan Testing**

In [73]:
# Split Data Training dan Testing (Rasio 80:20)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



---

### **Model Decision Tree**

In [7]:
# model DT
dt = DecisionTreeClassifier(random_state=42)

**Tunning Hyperparameter DT**

In [74]:
# mendefinisikan hyperparameter DT
dt_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [75]:
# menggunakan Validasi Grid Search Cross pada Decision Tree
# Validasi Grid Search Cross adalah teknik untuk menemukan hyperparameter terbaik
# pada suatu model dengan mencoba semua kombinasi hyperparameter yang mungkin

# membuat obyek GridSearchCV
dt_grid_search = GridSearchCV(estimator=dt, param_grid=dt_param_grid, cv=5, n_jobs=1, verbose=1)  # cv=5 menunjukkan cross-validation sebesar 5-fold

# Sesuaikan dt ke set training
dt_grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 126 candidates, totalling 630 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10, 15, 20, 30, 40],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             verbose=1)

In [76]:
# mencari parameter terbaik dari GridSearchCV
dt_best_params = dt_grid_search.best_params_

# mencari estimator terbaik
dt_best_estimator = dt_grid_search.best_estimator_

# mencari score terbaik
dt_best_score = dt_grid_search.best_score_

# print hasil parameter terbaik
print("Hyperparameter terbaik pada Decision Tree: ", dt_best_params)
print("Estimator terbaik:", dt_best_estimator)
print("Skor terbaik:", dt_best_score)

Hyperparameter terbaik pada Decision Tree:  {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Estimator terbaik: DecisionTreeClassifier(random_state=42)
Skor terbaik: 1.0


In [77]:
# Setelah mengetahui parameter terbaik, maka selanjutnya membuat model dan melatih model
# dengan parameter terbaik tersebut.

# model DT dengan parameter terbaik
dt_best_model_param = DecisionTreeClassifier(**dt_best_params)

# model DT dengan estimator terbaik
dt_best_model_est = dt_best_estimator

# model DT dengan score terbaik
dt_best_model_scr = DecisionTreeClassifier(random_state=42)


In [92]:
# Sesuaikan dt ke set training
dt_best_model_param.fit(X_train, y_train)
dt_best_model_scr.fit(X_train, y_train)


# memprediksi label set test
y_pred_dt_param = dt_best_model_param.predict(X_test)
y_pred_dt_est = dt_best_model_est.predict(X_test)
y_pred_dt_scr = dt_best_model_scr.predict(X_test)


# akurasi model dt_best_model
accuracy_dt_param = accuracy_score(y_test, y_pred_dt_param)
print("Akurasi Decision Tree dengan Parameter: {:.2f}".format(accuracy_dt_param))

accuracy_dt_est = accuracy_score(y_test, y_pred_dt_est)
print("Akurasi Decision Tree dengan Estimator: {:.2f}".format(accuracy_dt_est))

accuracy_dt_scr = accuracy_score(y_test, y_pred_dt_scr)
print("Akurasi Decision Tree dengan Score: {:.2f}".format(accuracy_dt_scr))

Akurasi Decision Tree dengan Parameter: 1.00
Akurasi Decision Tree dengan Estimator: 1.00
Akurasi Decision Tree dengan Score: 1.00




---

### **Model AdaBoost**

In [79]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning/Week 06/Tugas Praktikum/mushrooms.csv')

df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [80]:
# Seleksi fitur
X = df.drop('class', axis=1)  # fitur class di-drop karena hanya mendefinisikan jamur dapat dimakan atau tidak, axis=1 menandakan arah horizontal
y = df['class']  # fitur class sebagai target, karena fitur class di-drop, data 'p' dan 'e' di dalamnya tidak perlu di encode

# membuat model Label Encoder
le = LabelEncoder()

# encode fungsi/kolom
for col in X.columns:
  if X[col].dtype == 'object':
    X[col] = le.fit_transform(X[col])

# Cek jumlah fitur dan instance
print(X.shape)
X.info()
y.info()

(8124, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   cap-shape                 8124 non-null   int64
 1   cap-surface               8124 non-null   int64
 2   cap-color                 8124 non-null   int64
 3   bruises                   8124 non-null   int64
 4   odor                      8124 non-null   int64
 5   gill-attachment           8124 non-null   int64
 6   gill-spacing              8124 non-null   int64
 7   gill-size                 8124 non-null   int64
 8   gill-color                8124 non-null   int64
 9   stalk-shape               8124 non-null   int64
 10  stalk-root                8124 non-null   int64
 11  stalk-surface-above-ring  8124 non-null   int64
 12  stalk-surface-below-ring  8124 non-null   int64
 13  stalk-color-above-ring    8124 non-null   int64
 14  stalk-color-below-ring    812

In [81]:
# Split Data Training dan Testing (Rasio 80:20)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [86]:
# model AdaBoost
ada = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2), random_state=42)

**Tunning Hyperparameter AdaBoost**

In [87]:
# menggunakan Validasi Randomized Search Cross pada AdaBoost

ada_param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.1, 1.0],
    'algorithm': ['SAMME']
}

In [88]:
# Perform Randomized Search Cross Validation pada AdaBoost
# Randomized Search Cross Validation adalah teknik untuk menemukan hyperparameter terbaik
# pada suatu model dengan mencoba beberapa kombinasi hyperparameter yang dipilih secara acak
# dari distribusi yang ditentukan sebelumnya

ada_random_search = RandomizedSearchCV(ada, ada_param_grid, cv=5)  # 5-fold cross-validation

# train model
ada_random_search.fit(X_train, y_train)


RandomizedSearchCV(cv=5,
                   estimator=AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2),
                                                random_state=42),
                   param_distributions={'algorithm': ['SAMME'],
                                        'learning_rate': [0.01, 0.1, 1.0],
                                        'n_estimators': [50, 100, 200, 300]})

In [89]:
# mencari parameter terbaik dari RandomizedSearchCV
ada_best_params = ada_random_search.best_params_

# mencari estimator terbaik
ada_best_estimator = ada_random_search.best_estimator_

# mencari score terbaik
ada_best_score = ada_random_search.best_score_


# print hasil
print("Hyperparameter terbaik pada AdaBoost: ", ada_best_params)
print("Estimator terbaik:", ada_best_estimator)
print("Skor terbaik:", ada_best_score)

Hyperparameter terbaik pada AdaBoost:  {'n_estimators': 300, 'learning_rate': 1.0, 'algorithm': 'SAMME'}
Estimator terbaik: AdaBoostClassifier(algorithm='SAMME',
                   estimator=DecisionTreeClassifier(max_depth=2),
                   n_estimators=300, random_state=42)
Skor terbaik: 1.0


In [90]:
# Setelah mengetahui parameter terbaik, maka selanjutnya membuat model dan melatih model
# dengan parameter terbaik tersebut.

# model RF dengan parameter terbaik
ada_best_model = AdaBoostClassifier(**ada_best_params)

# model DT dengan estimator terbaik
ada_best_model_est = ada_best_estimator

# model DT dengan score terbaik
ada_best_model_scr = DecisionTreeClassifier(random_state=42)

In [91]:
# Sesuaikan ada ke set training
ada_best_model.fit(X_train, y_train)
ada_best_model_scr.fit(X_train, y_train)

# memprediksi label set test
y_pred_ada = ada_best_model.predict(X_test)
y_pred_ada_est = ada_best_model_est.predict(X_test)
y_pred_ada_scr = ada_best_model_scr.predict(X_test)

# akurasi model rf_best_model
accuracy_ada = accuracy_score(y_test, y_pred_ada)
print("Akurasi AdaBoost dengan Parameter: {:.2f}".format(accuracy_ada))

accuracy_ada_est = accuracy_score(y_test, y_pred_ada_est)
print("Akurasi AdaBoost dengan Estimator: {:.2f}".format(accuracy_ada_est))

accuracy_ada_scr = accuracy_score(y_test, y_pred_ada_scr)
print("Akurasi AdaBoost dengan Score: {:.2f}".format(accuracy_ada_scr))

Akurasi AdaBoost: 1.00
Akurasi AdaBoost: 1.00
Akurasi AdaBoost: 1.00


---

**Perbandingan Performa Algoritma DT dan AdaBoost**

In [93]:
# Evaluasi hasil pada set test
print("Akurasi Decision Tree dengan Parameter: {:.2f}".format(accuracy_dt_param))
print("Akurasi Decision Tree dengan Estimator: {:.2f}".format(accuracy_dt_est))
print("Akurasi Decision Tree dengan Score: {:.2f}".format(accuracy_dt_scr))

print("Akurasi AdaBoost dengan Parameter: {:.2f}".format(accuracy_ada))
print("Akurasi AdaBoost dengan Estimator: {:.2f}".format(accuracy_ada_est))
print("Akurasi AdaBoost dengan Score: {:.2f}".format(accuracy_ada_scr))

Akurasi Decision Tree dengan Parameter: 1.00
Akurasi Decision Tree dengan Estimator: 1.00
Akurasi Decision Tree dengan Score: 1.00
Akurasi AdaBoost dengan Parameter: 1.00
Akurasi AdaBoost dengan Estimator: 1.00
Akurasi AdaBoost dengan Score: 1.00
